<a href="https://colab.research.google.com/github/maybrg03/Laboratorio_RPC/blob/main/Laboratorio_RPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install -q grpcio grpcio-tools nbformat

In [48]:
%%bash

# CALCULADORA --------

cat > calculator.proto <<'PROTO'
syntax = "proto3";
package calc;

message Operands {
  double x = 1;
  double y = 2;
}

message Result {
  double value = 1;
  string error = 2;
}

service Calculator {
  rpc Add (Operands) returns (Result);
  rpc Sub (Operands) returns (Result);
  rpc Mul (Operands) returns (Result);
  rpc Div (Operands) returns (Result);
}
PROTO

cat > miner.proto <<'PROTO'
syntax = "proto3";
package miner;

message Empty {}

message TxID {
  int32 transaction_id = 1;
}

message ChallengeResp {
  int32 challenge = 1; // -1 if invalid tx
}

message StatusResp {
  int32 status = 1; // 0 resolved, 1 pending, -1 invalid
}

message SubmitReq {
  int32 transaction_id = 1;
  int32 client_id = 2;
  string solution = 3;
}

message SubmitResp {
  int32 code = 1; // 1 valid, 0 invalid, 2 already solved, -1 invalid tx
  string message = 2;
}

message WinnerResp {
  int32 winner = 1;
  string message = 2;
}

message SolutionResp {
  int32 status = 1; // 0 resolved, 1 pending, -1 invalid
  string solution = 2;
  int32 challenge = 3;
}
service Miner {
  rpc GetTransactionID (Empty) returns (TxID);
  rpc GetChallenge (TxID) returns (ChallengeResp);
  rpc GetTransactionStatus (TxID) returns (StatusResp);
  rpc SubmitChallenge (SubmitReq) returns (SubmitResp);
  rpc GetWinner (TxID) returns (WinnerResp);
  rpc GetSolution (TxID) returns (SolutionResp);
}
PROTO

cat > calculator_server.py <<'PY'
import grpc
from concurrent import futures
import time
import calculator_pb2
import calculator_pb2_grpc

class CalculatorServicer(calculator_pb2_grpc.CalculatorServicer):
    def Add(self, request, context):
        return calculator_pb2.Result(value=request.x + request.y)
    def Sub(self, request, context):
        return calculator_pb2.Result(value=request.x - request.y)
    def Mul(self, request, context):
        return calculator_pb2.Result(value=request.x * request.y)
    def Div(self, request, context):
        if request.y == 0:
            return calculator_pb2.Result(value=0.0, error="Division by zero")
        return calculator_pb2.Result(value=request.x / request.y)

def serve(port=50051):
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    calculator_pb2_grpc.add_CalculatorServicer_to_server(CalculatorServicer(), server)
    server.add_insecure_port(f'[::]:{port}')
    server.start()
    print(f"Calculator server started on port {port}")
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        server.stop(0)

if __name__ == "__main__":
    serve()
PY

cat > calculator_client.py <<'PY'
import grpc
import calculator_pb2
import calculator_pb2_grpc

def menu(stub):
    while True:
        print("\\n--- Calculadora RPC ---")
        print("1) Soma")
        print("2) Subtração")
        print("3) Multiplicação")
        print("4) Divisão")
        print("0) Sair")
        op = input("Escolha: ").strip()
        if op == "0":
            break
        if op not in ("1","2","3","4"):
            print("Opção inválida")
            continue
        try:
            x = float(input("x = "))
            y = float(input("y = "))
        except:
            print("Entrada inválida")
            continue
        req = calculator_pb2.Operands(x=x,y=y)
        if op == "1":
            r = stub.Add(req)
        elif op == "2":
            r = stub.Sub(req)
        elif op == "3":
            r = stub.Mul(req)
        elif op == "4":
            r = stub.Div(req)
        if r.error:
            print("Erro:", r.error)
        else:
            print("Resultado:", r.value)

def run(address="localhost:50051"):
    channel = grpc.insecure_channel(address)
    stub = calculator_pb2_grpc.CalculatorStub(channel)
    menu(stub)

if __name__ == "__main__":
    run()
PY


# MINERADOR -------

cat > miner_server.py <<'PY'
import grpc
from concurrent import futures
import miner_pb2
import miner_pb2_grpc
import threading
import random
import hashlib
import time

# Transaction record:
# { transaction_id: { 'challenge': int, 'solution': str or '', 'winner': int (clientID) } }

class MinerServicer(miner_pb2_grpc.MinerServicer):
    def __init__(self):
        self.lock = threading.Lock()
        self.tx_table = {}
        self.next_tx = 0
        # cria transação inicial
        self._create_new_tx(0)

    def _create_new_tx(self, txid):
        challenge = random.randint(1, 6)
        self.tx_table[txid] = {"challenge": challenge, "solution": "", "winner": -1}
        self.next_tx = txid + 1

    def GetTransactionID(self, request, context):
        with self.lock:
            for tid in sorted(self.tx_table.keys()):
                if self.tx_table[tid]['winner'] == -1:
                    return miner_pb2.TxID(transaction_id=tid)
            new_id = self.next_tx
            self._create_new_tx(new_id)
            return miner_pb2.TxID(transaction_id=new_id)

    def GetChallenge(self, request, context):
        tid = request.transaction_id
        with self.lock:
            if tid in self.tx_table:
                return miner_pb2.ChallengeResp(challenge=self.tx_table[tid]['challenge'])
            else:
                return miner_pb2.ChallengeResp(challenge=-1)

    def GetTransactionStatus(self, request, context):
        tid = request.transaction_id
        with self.lock:
            if tid not in self.tx_table:
                return miner_pb2.StatusResp(status=-1)
            return miner_pb2.StatusResp(status=0 if self.tx_table[tid]['winner'] != -1 else 1)

    def SubmitChallenge(self, request, context):
        tid = request.transaction_id
        client_id = request.client_id
        sol = request.solution
        with self.lock:
            if tid not in self.tx_table:
                return miner_pb2.SubmitResp(code=-1, message="Invalid transaction ID")
            record = self.tx_table[tid]
            if record['winner'] != -1:
                return miner_pb2.SubmitResp(code=2, message="Already solved")
            h = hashlib.sha1(sol.encode('utf-8')).hexdigest()
            target = '0' * record['challenge']
            if h.startswith(target):
                record['solution'] = sol
                record['winner'] = client_id
                new_id = self.next_tx
                self._create_new_tx(new_id)
                return miner_pb2.SubmitResp(code=1, message=f"Accepted (hash={h})")
            else:
                return miner_pb2.SubmitResp(code=0, message=f"Invalid solution (hash={h})")

    def GetWinner(self, request, context):
        tid = request.transaction_id
        with self.lock:
            if tid not in self.tx_table:
                return miner_pb2.WinnerResp(winner=-1, message="Invalid tx")
            winner = self.tx_table[tid]['winner']
            msg = "No winner yet" if winner == -1 else "Winner present"
            return miner_pb2.WinnerResp(winner=winner, message=msg)

    def GetSolution(self, request, context):
        tid = request.transaction_id
        with self.lock:
            if tid not in self.tx_table:
                return miner_pb2.SolutionResp(status=-1, solution="", challenge=0)
            rec = self.tx_table[tid]
            status = 0 if rec['winner'] != -1 else 1
            return miner_pb2.SolutionResp(status=status, solution=rec['solution'], challenge=rec['challenge'])

def serve(port=50052):
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=20))
    miner_pb2_grpc.add_MinerServicer_to_server(MinerServicer(), server)
    server.add_insecure_port(f'[::]:{port}')
    server.start()
    print(f"Miner server started on port {port}")
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        server.stop(0)

if __name__ == "__main__":
    serve()
PY

cat > miner_client.py <<'PY'
import grpc
import miner_pb2
import miner_pb2_grpc
import threading
import hashlib
import random
import string
import time

def try_solve(challenge, stop_event, found_holder):
    while not stop_event.is_set():
        nonce = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        h = hashlib.sha1(nonce.encode('utf-8')).hexdigest()
        if h.startswith('0' * challenge):
            found_holder['solution'] = nonce
            stop_event.set()
            return

def mine(stub, client_id, threads=4):
    txid = stub.GetTransactionID(miner_pb2.Empty()).transaction_id
    ch = stub.GetChallenge(miner_pb2.TxID(transaction_id=txid)).challenge
    if ch == -1:
        print("Invalid transaction")
        return
    print(f"Mining tx {txid} challenge={ch} ...")
    stop_event = threading.Event()
    found = {}
    workers = []
    for _ in range(threads):
        t = threading.Thread(target=try_solve, args=(ch, stop_event, found))
        t.start()
        workers.append(t)
    timeout = 30
    t0 = time.time()
    while not stop_event.is_set() and time.time() - t0 < timeout:
        time.sleep(0.5)
    if not stop_event.is_set():
        print("Timeout, no solution found in time.")
        stop_event.set()
    for w in workers:
        w.join()
    if 'solution' in found:
        sol = found['solution']
        print("Found solution:", sol)
        resp = stub.SubmitChallenge(miner_pb2.SubmitReq(transaction_id=txid, client_id=client_id, solution=sol))
        print("Submit response:", resp.code, resp.message)
    else:
        print("No solution found.")

def menu(stub, client_id):
    while True:
        print("\\n--- Miner Client ---")
        print("1) getTransactionID")
        print("2) getChallenge")
        print("3) getTransactionStatus")
        print("4) getWinner")
        print("5) getSolution")
        print("6) Mine (multithreaded)")
        print("0) Sair")
        op = input("Escolha: ").strip()
        if op == "0":
            break
        if op == "1":
            print(stub.GetTransactionID(miner_pb2.Empty()))
        elif op == "2":
            t = int(input("txid: "))
            print(stub.GetChallenge(miner_pb2.TxID(transaction_id=t)))
        elif op == "3":
            t = int(input("txid: "))
            print(stub.GetTransactionStatus(miner_pb2.TxID(transaction_id=t)))
        elif op == "4":
            t = int(input("txid: "))
            print(stub.GetWinner(miner_pb2.TxID(transaction_id=t)))
        elif op == "5":
            t = int(input("txid: "))
            print(stub.GetSolution(miner_pb2.TxID(transaction_id=t)))
        elif op == "6":
            th = int(input("Threads (ex: 4): ") or "4")
            mine(stub, client_id, threads=th)
        else:
            print("Opção inválida")

def run(address="localhost:50052", client_id=1):
    channel = grpc.insecure_channel(address)
    stub = miner_pb2_grpc.MinerStub(channel)
    menu(stub, client_id)

if __name__ == "__main__":
    run()
PY


In [49]:
%%bash
python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. calculator.proto
python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. miner.proto

ls -l *.py *.proto

-rw-r--r-- 1 root root  1190 Nov 11 16:16 calculator_client.py
-rw-r--r-- 1 root root  8082 Nov 11 16:16 calculator_pb2_grpc.py
-rw-r--r-- 1 root root  1826 Nov 11 16:16 calculator_pb2.py
-rw-r--r-- 1 root root   327 Nov 11 16:16 calculator.proto
-rw-r--r-- 1 root root  1139 Nov 11 16:16 calculator_server.py
-rw-r--r-- 1 root root  3000 Nov 11 16:16 miner_client.py
-rw-r--r-- 1 root root 11453 Nov 11 16:16 miner_pb2_grpc.py
-rw-r--r-- 1 root root  3028 Nov 11 16:16 miner_pb2.py
-rw-r--r-- 1 root root   982 Nov 11 16:16 miner.proto
-rw-r--r-- 1 root root  3927 Nov 11 16:16 miner_server.py


In [50]:
!nohup python calculator_server.py &> calc_server.log &
!sleep 2
!tail -n 10 calc_server.log

In [51]:
!python calculator_client.py

\n--- Calculadora RPC ---
1) Soma
2) Subtração
3) Multiplicação
4) Divisão
0) Sair
Escolha: 1
x = 3
y = 3
Resultado: 6.0
\n--- Calculadora RPC ---
1) Soma
2) Subtração
3) Multiplicação
4) Divisão
0) Sair
Escolha: 2
x = 3
y = 3
Resultado: 0.0
\n--- Calculadora RPC ---
1) Soma
2) Subtração
3) Multiplicação
4) Divisão
0) Sair
Escolha: 3
x = 3
y = 3
Resultado: 9.0
\n--- Calculadora RPC ---
1) Soma
2) Subtração
3) Multiplicação
4) Divisão
0) Sair
Escolha: 4
x = 3
y = 3
Resultado: 1.0
\n--- Calculadora RPC ---
1) Soma
2) Subtração
3) Multiplicação
4) Divisão
0) Sair
Escolha: 0


In [52]:
!nohup python miner_server.py &> miner_server.log &
!sleep 2
!tail -n 10 miner_server.log

In [53]:
!python miner_client.py

\n--- Miner Client ---
1) getTransactionID
2) getChallenge
3) getTransactionStatus
4) getWinner
5) getSolution
6) Mine (multithreaded)
0) Sair
Escolha: 1

\n--- Miner Client ---
1) getTransactionID
2) getChallenge
3) getTransactionStatus
4) getWinner
5) getSolution
6) Mine (multithreaded)
0) Sair
Escolha: 2
txid: 1
challenge: -1

\n--- Miner Client ---
1) getTransactionID
2) getChallenge
3) getTransactionStatus
4) getWinner
5) getSolution
6) Mine (multithreaded)
0) Sair
Escolha: 3
txid: 1
status: -1

\n--- Miner Client ---
1) getTransactionID
2) getChallenge
3) getTransactionStatus
4) getWinner
5) getSolution
6) Mine (multithreaded)
0) Sair
Escolha: 4
txid: 1
winner: -1
message: "Invalid tx"

\n--- Miner Client ---
1) getTransactionID
2) getChallenge
3) getTransactionStatus
4) getWinner
5) getSolution
6) Mine (multithreaded)
0) Sair
Escolha: 4
txid: 0
winner: -1
message: "No winner yet"

\n--- Miner Client ---
1) getTransactionID
2) getChallenge
3) getTransactionStatus
4) getWinner
5) 

In [54]:
!pkill -f calculator_server.py || true
!pkill -f miner_server.py || true

^C
^C
